In [2]:
# まずは業種を取得する。
# 静的なサイトなのでpyqueryを用いる。
from pyquery import PyQuery as pq

q = pq('https://kabutan.jp/stock/?code=7203')
sector = q.find('#stockinfo_i2 a')[0].text#table.kobetsu_data_table2 a')[0].text
print(sector)

輸送用機器


In [ ]:
# 今度は最小単位を取ってみる。
# seleniumを練習するため動的なサイトである株mapというサイトから。
from selenium import webdriver as wd

url = 'https://jp.kabumap.com/servlets/kabumap/Action' \
        '?SRC=basic/top/base&codetext=7203'
driver = wd.Chrome()
driver.get(url)

unit = driver.find_element_by_css_selector('#minUnit').text

print(unit)

In [69]:
    code = 1319
    
    url = 'https://kabutan.jp/stock/?code={}'.format(code)
    
    q = pq(url)

    if len(q.find('.si_i1_dl1')) == 0:
        None
    else:
        print(len(q.find('.si_i1_dl1')))


1


In [89]:
    code = 1306
    
    url = 'https://kabutan.jp/stock/?code={}'.format(code)
    
    q = pq(url)        
    
    unit_str = q.find('#stockinfo_i2 dd')[1].text
    unit = int(unit_str.split('株')[0].replace(',',''))
    
    unit


10

In [86]:
# 関数定義
from pyquery import PyQuery as pq
import time
import sqlite3

def get_brand(code):
    url = 'https://kabutan.jp/stock/?code={}'.format(code)
    
    q = pq(url)

    if len(q.find('.si_i1_dl1')) == 0:
        return None

    try:
        name = q.find('#kobetsu_right h3')[0].text
        short_name = q.find('.topicpath span')[1].text.split('(', 1)[0]
        market = q.find('.market')[0].text
        unit_str = q.find('#stockinfo_i2 dd')[1].text
        unit = int(unit_str.split('株')[0].replace(',',''))
        sector = q.find('#stockinfo_i2 a')[0].text
    except ValueError:
        return None
    
    return code, name, short_name, market, unit, sector

def brands_generator(code_range):
    for code in code_range:
        brand = get_brand(code)
        if brand:
            yield brand
        time.sleep(1)
        
def insert_brand_to_db(db_file_name, code_range):
    conn = sqlite3.connect(db_file_name)
    with conn:
        sql = 'INSERT INTO brands(code, name, short_name, market, unit, sector) VALUES(?,?,?,?,?,?)'
        conn.executemany(sql, brands_generator(code_range))


In [96]:
insert_brand_to_db('kabu_python.sqlite3', range(7000, 9999))